<a href="https://colab.research.google.com/github/roniadmon/ML/blob/master/rotem_error2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
some_tensor = torch.rand(1,9,9)
print(some_tensor.shape)
some_tensor = some_tensor.repeat(3,1,1) ## <--- repeat 
print(some_tensor.shape)

torch.Size([1, 9, 9])
torch.Size([3, 9, 9])


In [ ]:
import torch
from torch.utils.data import TensorDataset,DataLoader
import numpy as np
import os
from scipy import stats

os.chdir(os.getcwd())
path = 'data/'
fieldname = '_19790101-20190228.npy'
x1_arr = np.load(path+'z1000'+fieldname) # geopotential height data (9*9 resolution)
x2_arr = np.load(path+'pv300'+fieldname) # potential vorticity data (9*9 resolution)

x1_arr = stats.zscore(x1_arr) # normalize
x2_arr = stats.zscore(x2_arr)
y_arr = np.load(path+'rain_basin_19790101-20190228.npy') # rain data

print(x1_arr.shape, x2_arr.shape, y_arr.shape)

tensor_x = torch.Tensor(np.stack([x1_arr, x1_arr, x2_arr],axis=1)) # join z and pv data
tensor_y = torch.Tensor(y_arr)

forecast_dataset = TensorDataset(tensor_x,tensor_y) # creates a dataset based on tensors
# forecast_dataset2 = []
# for j in forecast_dataset:
#   forecast_dataset2.append( ( j[0].reshape((2, 9, 9)) , j[1] ) ) # Rotem: ask about order of 162
training_ds, validation_ds = torch.utils.data.random_split(forecast_dataset, [2195,1464])
training_dataloader = DataLoader(training_ds,batch_size=200,shuffle=True)
valid_dataloader = DataLoader(validation_ds,batch_size=200)



(3659, 9, 9) (3659, 9, 9) (3659, 4)


In [ ]:
for batch_x,batch_y in training_dataloader:
  print(batch_x.shape, batch_y.shape)

torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([195, 3, 9, 9]) torch.Size([195, 4])


In [ ]:
for batch_x,batch_y in valid_dataloader:
  print(batch_x.shape, batch_y.shape)

torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([200, 3, 9, 9]) torch.Size([200, 4])
torch.Size([64, 3, 9, 9]) torch.Size([64, 4])


for batch_x,batch_y in valid_dataloader:
  print(batch_x.shape, batch_y.shape)

In [ ]:
import torchvision.models as models
pretrained_model = models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


In [ ]:
pretrained_model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
for x,y in training_dataloader:
    print(x.shape,y.shape)
    break

torch.Size([200, 3, 9, 9]) torch.Size([200, 4])


In [ ]:
pretrained_model(x).shape #input size here doesn't fit

RuntimeError: ignored

In [ ]:
from model import Net
net = Net()

In [ ]:
net(x).shape

RuntimeError: ignored

how to use the pre-trained model weights?
we need to modify the state_dict of the model. this is dictionary that maps between a layer name to the actual weights

In [ ]:
for key in net.state_dict().keys():
    print(key)

copy the parts of the pre-trained model to your models state dict, and use load_state_dict to actually update the model. now you are ready to train

In [ ]:
pretrained_dict = pretrained_model.state_dict()
state_dict = net.state_dict()

In [ ]:
for key in state_dict.keys():
    if 'features' not in key:
        continue
    if key in pretrained_dict.keys():
        state_dict[key] = pretrained_dict[key]
net.load_state_dict(state_dict)